<h1 style="color:#987753"><center>============= IMPORT DEPANDENCIES =============</center></h1>

In [ ]:
import tensorflow as tf
import numpy as np
from random import shuffle
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt
import time
import random
from datetime import datetime
from datetime import date
from random import randint
from tensorflow.keras.callbacks import TensorBoard
import pandas as pd

<h1 style="color:#987753"><center>============= UTILS FUNCTION =============</center></h1>

In [ ]:
global ID
ID = random.randint(1,1000000)

In [ ]:
def getTime():
    
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S")
    today = str(date.today()).split('-')
    today = today[2]+"_"+today[1]+"_"+today[0]
    runTime = ("%s_%s" %(today,current_time))
    return runTime


In [ ]:
def logName():
    
    NAME = "ModelCifar10_MLP_"+str(ID)+"_"+getTime()
    return NAME

In [ ]:
def hyperParams():

    modelParams = {"Layers":[{"neurons":248, "activation":"sigmoid","dropOut":False},
                    {"neurons":145, "activation":"sigmoid", "dropOut":False},
                    {"neurons":73, "activation":"relu", "dropOut":False},
                    {"neurons":10, "activation":"softmax", "dropOut":False}],
                   "loss":"categorical_crossentropy",
                   "optimizer":"Adam",
                   "metrics":"accuracy",
                   "epochs":72,
                   "batchSize":450
                 }
    return modelParams

params = hyperParams()

In [ ]:
def humainesLabels(position):
    labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
    return labels[position]

In [ ]:
def humanizeTime(secs):
    mins, secs = divmod(secs, 60)
    hours, mins = divmod(mins, 60)
    return ('RUN TIME : %02d:%02d:%02d' % (hours, mins, secs))

In [ ]:
def loadData():
    
    (trainImages, trainLabels), (testImages, testLabels) = cifar10.load_data()
    trainImages, testImages = trainImages / 255.0, testImages / 255.0
    return trainImages, testImages, trainLabels, testLabels

(trainImages, testImages, trainLabels, testLabels) = loadData()

print("\t- NB TRAIN SAMPLES %s WIDTH %s HEIGHT %s COLOR %s" % (trainImages.shape))
print("\t- NB TEST SAMPLES %s WIDTH %s HEIGHT %s COLOR %s" % (testImages.shape))

In [ ]:
def plotFigure(images=False, labels=False, nbSamples=0, size=10):
    
    plt.figure(figsize=(size, size))
    for i in range(nbSamples):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
       
        plt.xlabel(humainesLabels(labels[i][0]))
    plt.show()
    
plotFigure(trainImages, trainLabels, nbSamples=25)

<h1 style="color:#987753"><center>============= INIT MODEL =============</center></h1>

In [ ]:
def oneHoteEncoding(trainLabels, testLabels):
    
    trainLabels = tf.keras.utils.to_categorical(trainLabels)
    testLabels = tf.keras.utils.to_categorical(testLabels)
    
    return trainLabels, testLabels

trainLabels, testLabels = oneHoteEncoding(trainLabels, testLabels)

In [ ]:
def createModel(params):
    
    model = Sequential()

    model.add(Flatten(input_shape=(32, 32, 3)))
    for layers in params['Layers']:

        model.add(Dense(layers['neurons'], activation=layers['activation']))
        if layers['dropOut'] != False:
            model.add(Dropout(layers['dropOut']))

    model.compile(loss=params['loss'],
                   optimizer=params['optimizer'],
                   metrics=['accuracy'])
    
    model.summary()
    return model

model = createModel(params)

<h1 style="color:#987753"><center>============= TRAIN MODEL =============</center></h1>

In [ ]:
def reshapeInput(trainImages, testImages):
    
    trainImages.reshape(trainImages.shape[0], trainImages.shape[1] * trainImages.shape[2] * trainImages.shape[3]) #
    testImages.reshape(testImages.shape[0], testImages.shape[1] * testImages.shape[2] * testImages.shape[3]) #
    
    return trainImages, testImages

trainImages, testImages = reshapeInput(trainImages, testImages)

In [ ]:
def fitModel(model, params):
    
    start_time = time.time()
    tensorboard = TensorBoard(log_dir='D:\logsProject\{}'.format(logName()))
    ################################ CALCULATE RUNTIME - START ###############################
    model.fit(trainImages, trainLabels,
                       epochs=params['epochs'],
                       batch_size=params['batchSize'],
                       validation_data=(testImages, testLabels), verbose=1, callbacks=[tensorboard])
    ################################ CALCULATE RUNTIME - END ###############################
    print(humanizeTime(time.time() - start_time))
    return model, humanizeTime(time.time() - start_time)

model, runTime = fitModel(model, params)

<h1 style="color:#987753"><center>============= EVALUATE MODEL =============</center></h1>

In [ ]:
def evaluateModel(model):
    
    return model.evaluate(testImages, testLabels, batch_size=50)

resultEV = evaluateModel(model)
resultEV

<h1 style="color:#987753"><center>============= PREDICT =============</center></h1>

In [ ]:
def predictionPercent(model, testImages, testLabels):
    
    prediction = model.predict(testImages)
    predicted = []
    for i in range(len(prediction)):
        if np.argmax(testLabels[i]) == np.argmax(prediction[i]):
            predicted.append(1)
        else:
            predicted.append(0)
    print("Predicted precent {} %".format((sum(predicted)/(len(predicted)) * 100)))
    result = (sum(predicted)/len(predicted)) * 100
    
    return prediction, result

prediction, result = predictionPercent(model, testImages, testLabels)

In [ ]:
def plotPredictedResult(prediction, testImages, testLabels):
    
    plt.figure(figsize=(15, 15))
    for i in range(20):
        plt.subplot(5,7,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(testImages[i])
        plt.xlabel("True: "+humainesLabels(np.argmax(testLabels[i]))+"\n\n\n")
        plt.title("\nPredicted: "+humainesLabels(np.argmax(prediction[i])))
    plt.show()
    
plotPredictedResult(prediction, testImages, testLabels)

<h1 style="color:#987753"><center>============= SAVE MODEL =============</center></h1>

In [ ]:
def saveModel(model):
    
    now = datetime.now()
    current_time = now.strftime("%H-%M-%S")
    today = str(date.today()).split('-')
    today = today[2]+"_"+today[1]+"_"+today[0]
    runTime = ("%s_%s" %(today,current_time))
    model.save('model_run_'+runTime)
    

In [ ]:
def generateExcelParams(params, resultEV, runTime):
    
    pass
    params['idExec'] = ID
    params['accuracyResult'] = resultEV[1]
    params['lossResult'] = resultEV[0]
    params['excTime'] = runTime.split(' ')[3]
    df = pd.DataFrame(params, columns=params.keys())
    
    df.to_excel(r'D:\logsProject\ModelParams_'+getTime()+"_"+str(ID)+'.xlsx', index = None, header=True)
    return df

jsonDf = generateExcelParams(params, resultEV, runTime)